In [1]:
# ─── PARSE INPUT ────────────────────────────────────────────────────────────────

import re

with open('input') as file:
    lines = file.read()

lines_ranges, lines_your_ticket, lines_nearby_tickets = lines.split('\n\n')

# ─── RANGES
ranges_dict = dict()
for string in lines_ranges.splitlines():
    name, numbers = string.split(':')
    name = name.replace(' ','')
    match = re.findall('\d+', numbers)
    ranges_dict.update({name: list(map(int, match))})

# ─── MY TICKET 
your_ticket = list(map(int, re.findall('\d+', lines_your_ticket)))

# ─── NEARBY TICKETS 
nearby_tickets = []
for line in lines_nearby_tickets.splitlines():
    match = re.findall('\d+', line)
    if match:
        nearby_tickets.append(list(map(int, match)))

In [2]:
# ─── CHALLENGE 1 ────────────────────────────────────────────────────────────────

def number_in_any_range(number):
    # check, if number is within one or more ranges
    number_valid = False
    for value in ranges_dict.values():
        if value[0] <= number <= value[1] or value[2] <= number <= value[3]:
            number_valid = True
            # break
    return number_valid

error_rate = 0
nearby_tickets_valid = []
for ticket in nearby_tickets:
    ticket_valid = True
    for number in ticket:
        if not number_in_any_range(number):
            # one number on ticket is outside of every single range
            error_rate += number
            ticket_valid = False
    if ticket_valid:
        nearby_tickets_valid.append(ticket)

print('result 1: ', error_rate)

result 1:  20058


In [3]:
# ─── CHALLENGE 2 ────────────────────────────────────────────────────────────────

from math import prod

field_to_name_counter = {i: dict() for i in range(20)}
field_to_name_solution = dict()

def count_valid_connections(number, field):
    # check if number is within one or more ranges
    for key, value in ranges_dict.items():
        if value[0] <= number <= value[1] or value[2] <= number <= value[3]:
            field_to_name_counter[field].update({key: field_to_name_counter[field].get(key, 0) + 1})

# for every field and every ticket, count, how often the fieldid->name
# relation is valid (is withing allowed range) 
for field in range(20):
    for ticket in nearby_tickets_valid:
        number = ticket[field]
        count_valid_connections(number, field)

def filter_counter(item):
    key, value = item
    condition1 = value == len(nearby_tickets_valid)
    condition2 = key not in field_to_name_solution.values()
    return all([condition1, condition2])

for _ in range(20):
    # assign field_to_name relation to solution, when there is only one 
    # valid assignment left
    for key, ct in field_to_name_counter.items():
        if len(ct) == 1:
            field_to_name_solution.update({key: list(ct.keys())[0]})

    # remove solved field name from field_to_name_counter
    field_to_name_counter_new = dict()
    for key, ct in field_to_name_counter.items():
        counter_new = dict(filter(filter_counter, ct.items()))
        if len(counter_new) > 0:
            field_to_name_counter_new[key] = counter_new
    field_to_name_counter = field_to_name_counter_new

# multiply all ticket values with field name starting with departure
solution = prod([your_ticket[key] for key, value in field_to_name_solution.items() if value.startswith('departure')])
print('result 2: ', solution)

result 2:  366871907221
